### 1. Import the data set you exported in Exercise 4.9 into a new Jupyter notebook. This should be the merged version of your data that contains the new customers column.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\esgib\Desktop\Alina\Course 4\Instacart Basket Analysis'

In [3]:
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared data', 'Orders_products_all.pkl'))

### 2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.

Answer: As I didn't merge customer informations like custoemr name or surname, therefore the data can't be used to track individual person

### 3. The Instacart officers are interested in comparing customer behavior in different geographic areas. Create a regional segmentation of the data. You’ll need to create a “Region” column based on the “State” column from your customers data set.
- Use the region information in this Wikipedia article to create your column (you only need to create regions, not divisions).
- Determine whether there’s a difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.)

In [4]:
#checkign for mixed datatypes

for col in df.columns.tolist(): 
    weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1) 
    if len (df[weird]) > 0: print (col)

In [5]:
#creating region lists

Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana' 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [6]:
#assigning again regions to each order

df.loc[df['state'].isin(Northeast), 'region'] = 'Northeast'
df.loc[df['state'].isin(Midwest), 'region'] = 'Midwest'
df.loc[df['state'].isin(South), 'region'] = 'South'
df.loc[df['state'].isin(West), 'region'] = 'West'

In [7]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,department_id,prices,gender,state,age,nr_dependants,fam_status,income,region
0,2,33120,1,1,202279,3,5,9,8.0,Organic Egg Whites,16.0,11.3,Male,Idaho,57,3,married,98119,West
1,2,28985,2,1,202279,3,5,9,8.0,Michigan Organic Kale,4.0,13.4,Male,Idaho,57,3,married,98119,West
2,2,9327,3,0,202279,3,5,9,8.0,Garlic Powder,13.0,3.6,Male,Idaho,57,3,married,98119,West
3,2,45918,4,1,202279,3,5,9,8.0,Coconut Butter,13.0,8.4,Male,Idaho,57,3,married,98119,West
4,2,30035,5,0,202279,3,5,9,8.0,Natural Sweetener,13.0,13.7,Male,Idaho,57,3,married,98119,West


In [8]:
df['region'].value_counts()

South        10801195
West          8300141
Midwest       6346809
Northeast     5728144
Name: region, dtype: int64

In [9]:
# create new column 'spend_mean' with average prices
df['spend_mean'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [10]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,department_id,prices,gender,state,age,nr_dependants,fam_status,income,region,spend_mean
0,2,33120,1,1,202279,3,5,9,8.0,Organic Egg Whites,16.0,11.3,Male,Idaho,57,3,married,98119,West,8.618889
1,2,28985,2,1,202279,3,5,9,8.0,Michigan Organic Kale,4.0,13.4,Male,Idaho,57,3,married,98119,West,8.618889
2,2,9327,3,0,202279,3,5,9,8.0,Garlic Powder,13.0,3.6,Male,Idaho,57,3,married,98119,West,8.618889
3,2,45918,4,1,202279,3,5,9,8.0,Coconut Butter,13.0,8.4,Male,Idaho,57,3,married,98119,West,8.618889
4,2,30035,5,0,202279,3,5,9,8.0,Natural Sweetener,13.0,13.7,Male,Idaho,57,3,married,98119,West,8.618889


In [11]:
df['spend_mean'].dtype

dtype('float64')

In [12]:
df['spend_mean'].value_counts().sum()

32433030

In [13]:
# Creating 'spend_flag' value and assigning the flag

df.loc[df['spend_mean'] <10, 'spend_flag'] = 'Low spender'
df.loc[df['spend_mean'] >= 10, 'spend_flag'] = 'High spender'

In [14]:
df['spend_flag'].value_counts()

Low spender     31797576
High spender      635454
Name: spend_flag, dtype: int64

In [15]:
region_spending_flag = pd.crosstab(df['region'], df['spend_flag'], dropna = False)

In [16]:
region_spending_flag

spend_flag,High spender,Low spender
region,,
Midwest,126253,6220556
Northeast,108342,5619802
South,210180,10591015
West,160807,8139334


In all regions there is aproximately same distribution: 98 % of low spenders and 2% of high spenders. However numericaly in the South region there is the highest number of high spenders, the lowest is in Northwest. 


### 4. The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [17]:
#customer_id(order_id.count)
df['total_orders'] = df.groupby(['user_id'])['order_id'].transform(np.sum)

In [18]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,...,gender,state,age,nr_dependants,fam_status,income,region,spend_mean,spend_flag,total_orders
0,2,33120,1,1,202279,3,5,9,8.0,Organic Egg Whites,...,Male,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0
1,2,28985,2,1,202279,3,5,9,8.0,Michigan Organic Kale,...,Male,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0
2,2,9327,3,0,202279,3,5,9,8.0,Garlic Powder,...,Male,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0
3,2,45918,4,1,202279,3,5,9,8.0,Coconut Butter,...,Male,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0
4,2,30035,5,0,202279,3,5,9,8.0,Natural Sweetener,...,Male,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0


In [19]:
#creating 'max_order' column and storing max order number
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [20]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,...,state,age,nr_dependants,fam_status,income,region,spend_mean,spend_flag,total_orders,max_order
0,2,33120,1,1,202279,3,5,9,8.0,Organic Egg Whites,...,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0,8
1,2,28985,2,1,202279,3,5,9,8.0,Michigan Organic Kale,...,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0,8
2,2,9327,3,0,202279,3,5,9,8.0,Garlic Powder,...,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0,8
3,2,45918,4,1,202279,3,5,9,8.0,Coconut Butter,...,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0,8
4,2,30035,5,0,202279,3,5,9,8.0,Natural Sweetener,...,Idaho,57,3,married,98119,West,8.618889,Low spender,145532043.0,8


In [21]:
#assigning activity flag
df.loc[df['max_order'] <5, 'activity_flag'] = 'Low_activity'
df.loc[df['max_order'] >= 5, 'activity_flag'] = 'High_activity'

In [22]:
df['activity_flag'].value_counts()

High_activity    30991542
Low_activity      1441488
Name: activity_flag, dtype: int64

In [23]:
df_ha = df[df['activity_flag'] == 'High_activity']

In [24]:
df_ha.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,...,age,nr_dependants,fam_status,income,region,spend_mean,spend_flag,total_orders,max_order,activity_flag
0,2,33120,1,1,202279,3,5,9,8.0,Organic Egg Whites,...,57,3,married,98119,West,8.618889,Low spender,145532043.0,8,High_activity
1,2,28985,2,1,202279,3,5,9,8.0,Michigan Organic Kale,...,57,3,married,98119,West,8.618889,Low spender,145532043.0,8,High_activity
2,2,9327,3,0,202279,3,5,9,8.0,Garlic Powder,...,57,3,married,98119,West,8.618889,Low spender,145532043.0,8,High_activity
3,2,45918,4,1,202279,3,5,9,8.0,Coconut Butter,...,57,3,married,98119,West,8.618889,Low spender,145532043.0,8,High_activity
4,2,30035,5,0,202279,3,5,9,8.0,Natural Sweetener,...,57,3,married,98119,West,8.618889,Low spender,145532043.0,8,High_activity


In [25]:
df_ha.shape

(30991542, 24)

In [26]:
df_ha.to_pickle(os.path.join(path, '02 Data', 'Prepared data', 'High_act_cust.pkl'))

In [27]:
df.to_pickle(os.path.join(path, '02 Data', 'Prepared data', 'Orders_products_all.pkl'))